In [1]:
import sys
print(sys.version)

3.4.5 |Continuum Analytics, Inc.| (default, Jul  5 2016, 14:53:07) [MSC v.1600 64 bit (AMD64)]


In [2]:
import os
import pickle

# input 
familyName = "eggnog"
generation = "main"
treeNumber = "all"

data_directory = "tracelogs_analysis_simplify/Selected/"+familyName+"/"+generation+"/"+ treeNumber + '/' # make sure the last character is '/'
in_tag = familyName + "_0.8"
in_parseFirstPar = True # keep first parameter or not
in_window = 1

outputPath = "output/GSA/" + in_tag + "-2/" + data_directory.split('/')[-2] + "/" # MIKE: no 'l' at the end. It is ridiculous!!

In [3]:
# def fp_removeDuplicateAPI(featureTrace): # remove duplicate api if continuously occur
#     result = []
#     lastAPI = ""
#     for api in featureTrace:
#         if lastAPI != api: # find new api
#             result.append(api)
#             lastAPI = api
#     return result

# def fp_removeUnwantedAPI(featureTrace): # remove unwanted api
#     result = []
#     unwanted_api = {'CloseHandle', 'OpenThread', 'RegOpenKey', 'RegCloseKey'}
#     frequently_used_lib = {'imm32', 'lpk', 'gdi32', 'kernel32', 'ntdll', 'user32', 'comctl32', 'advapi64'}

#     for api in featureTrace:
#         API = api.split('#')[0]
        
#         if API == "LoadLibrary": # api is LoadLibrary
#             libName = api.split("@")[2]
#             if libName not in frequently_used_lib: # found new library, add it into lib_set and result_Hooklog
#                 result.append(api)
#                 frequently_used_lib.update(libName)
                
#         elif API not in unwanted_api: # api not unwanted
#             result.append(api)
            
#     return result

# def et_removeDuplicateAPI(execTrace):
#     result = []
#     lastAPI = ""
#     for trace in execTrace:
#         timeStamp = trace[0]
#         api = trace[1]
#         if lastAPI != api: # find new api
#             result.append((timeStamp, api))
#             lastAPI = api
#     return result

# def et_removeUnwantedAPI(execTrace): # remove unwanted api
#     result = []
#     unwanted_api = {'CloseHandle', 'OpenThread', 'RegOpenKey', 'RegCloseKey'}
#     frequently_used_lib = {'imm32', 'lpk', 'gdi32', 'kernel32', 'ntdll', 'user32', 'comctl32', 'advapi64'}

#     for trace in execTrace:
#         timeStamp = trace[0]
#         api = trace[1]
        
#         API = api.split('#')[0]
        
#         if API == "LoadLibrary": # api is LoadLibrary
#             libName = api.split("@")[2]
#             if libName not in frequently_used_lib: # found new library, add it into lib_set and result_Hooklog
#                 result.append((timeStamp, api))
#                 frequently_used_lib.update(libName)
                
#         elif API not in unwanted_api: # api not unwanted
#             result.append((timeStamp, api))
            
#     return result

In [4]:
# 1.Get a dict of feature profiles (out: a hkName list, a featureProfile dict)
%run FeatureTrace.ipynb
FeatTrace = FeatureTrace

traceName_li = list(filter(lambda f:f.endswith('.trace.hooklog'), os.listdir(data_directory))) # hooklog Name List

fp_dict = dict()
execTrace_dict = dict()

for f in traceName_li:
    featureTrace = FeatTrace(data_directory + f).getTrace_noContainTS()
#     featureTrace = fp_removeDuplicateAPI(featureTrace)
#     featureTrace = fp_removeUnwantedAPI(featureTrace)
    fp_dict[f] = featureTrace
    
    execTrace = FeatTrace(data_directory + f).getTrace_containTS()
#     execTrace = et_removeDuplicateAPI(execTrace)
#     execTrace = et_removeUnwantedAPI(execTrace)
    execTrace_dict[f] = execTrace

In [7]:
# 2.Get a dict of pairwise alignment result 
# (in: a featureProfile dict; out: alignmentProfile dict, alingmnt baseline)

# %run Alignment3.ipynb
%run Alignment_Fast3.ipynb

BASE = traceName_li[0] # randomly pick a BASE trace log
# BASE = "008e76aafe9bd3132293816e6b0ac8e68cea5445c47065e53d7028ddbe375843_3272.trace.hooklog" # specified base
fpBASE = fp_dict[BASE]
print(BASE)
align_dict = {hk:pairwise_NW( fpBASE, fp_dict[hk], 2, -1, -3, 1)[2] for hk in traceName_li } # pairwise all

008e76aafe9bd3132293816e6b0ac8e68cea5445c47065e53d7028ddbe375843_3272.trace.hooklog


In [8]:
# 3. Get matchMatrix, gapSeqList
# (in: alignmentProfile dict; out: matchMatrix, gapSeqList)
%run StructMatchGap3.ipynb

data = structMatchGap(align_dict, BASE)
matchMatrix = data[0]
gapSeqList = data[1]

In [9]:
# 4. Get stageMatrix 
# (in: matchMatrix, gapSeqList; out: stageMatrix)
% run StageMatrix.ipynb

stageMatrixResult = stageMatrix(matchMatrix, gapSeqList)

In [10]:
#  5. Get Motif Elements
# (in: stageMatrixResult, BASE; out: Motif object)
% run Motif.ipynb

Motif = Motif(stageMatrixResult, BASE)

In [11]:
######  Garbage Code  -->>>  Work around only!   #####
###  WJ:  I don't know why do this!?

#with open('docs/label_dict-m2t-t2m-2014.pickle', 'rb') as f:
#    label_dict = pickle.load(f)[1] #(proc_list,proc_dict,label_list, label_dict)
#label_dict = {k[:-len('.trace.hooklog')]:label_dict[k] for k in label_dict}

# MIKE: 20170808, I comment it. Need to come back again.
label_dict = dict()

In [13]:
## Drawing stages graphs
## output into 'outputPath'
% run OutputStageBarchart3.ipynb

if not os.path.isdir(outputPath): os.makedirs(outputPath)
outputStage = OutputStage( stageMatrixResult, outputPath, BASE, Motif ) # output stages graphs
OutputMotiGraph(stageMatrixResult, BASE, fp_dict, outputStage, outputPath, Motif, label_dict) # output motif graphs

In [14]:
## Output common motif analysis
# output into 'outputPath'
% run CommonMotifAnalysis3.ipynb
execTrace_dict = fp_dict
motiDict = MotiDict(stageMatrixResult, Motif, execTrace_dict, outputStage)
comMotif = motiDict.getComMoti()
# comResAnalysis = ComResAnalysis(comMotif)

In [15]:
% run OutputCommonMotif3.ipynb
OutputCsv(outputPath, comMotif, stageMatrixResult, Motif, BASE, execTrace_dict, outputStage)
# OutputComMotiAnaly(comResAnalysis, outputPath)